<a href="https://colab.research.google.com/github/Karenrojas20/Seguimiento_gestion_contractual/blob/main/00_SECOP_Atenea_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Carga de librerías extra e insumos**

In [ ]:
!pip install -q rapidfuzz unidecode python-dateutil
import os
import re
import time
import math
import json
import random
import unicodedata
from datetime import datetime
from typing import Optional, List, Dict, Any

import numpy as np
import pandas as pd
import requests
from rapidfuzz import process, fuzz
from dateutil.relativedelta import relativedelta
from collections import Counter
import ast
from tqdm.auto import tqdm

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

from google.colab import files

In [ ]:
# ==========================
# Cargar Insumos_Codigo00.xlsx desde tu PC
# ==========================

print("Sube el archivo Insumos_Codigo00.xlsx")
uploaded = files.upload()  # Seleccionas Insumos_Codigo00.xlsx

if not uploaded:
    raise RuntimeError("No se subió ningún archivo. Debes subir Insumos_Codigo00.xlsx")
secop_filename = list(uploaded.keys())[0]

Lista_Nombres_Genero = pd.read_excel(secop_filename, sheet_name= "Nombre-Genero")
print("Lista_Nombres_Genero cargado:", Lista_Nombres_Genero.shape)

tab_proyectos = pd.read_excel(secop_filename, sheet_name= "Proyectos-Meta")
print("tab_proyectos cargado:", tab_proyectos.shape)

tab_pospre = pd.read_excel(secop_filename, sheet_name= "Pospre")
print("tab_pospre:", tab_pospre.shape)

tab_tc = pd.read_excel(secop_filename, sheet_name= "Tipo de contrato")
print("tab_tc cargado:", tab_tc.shape)

Sube el archivo Insumos_Codigo00.xlsx


Saving Insumos_Codigo00 (1).xlsx to Insumos_Codigo00 (1) (1).xlsx
Lista_Nombres_Genero cargado: (1479, 2)
tab_proyectos cargado: (36, 5)
tab_pospre: (19, 2)
tab_tc cargado: (13, 2)


## **Imports, sesión y parámetros globales**

In [ ]:
# Nombre del archivo de salida
EXCEL_FILE = "SECOP_ATENEA_TOTAL.xlsx"

# --------- Parámetros de negocio ---------
ENTITY_NAME = "AGENCIA DISTRITAL PARA LA EDUCACIÓN SUPERIOR, LA CIENCIA Y LA TECNOLOGÍA, ATENEA"
ENTITY_NIT  = "901508361"

# # ---------------------------------------------------
# # --------- Modificar fecha de inicio según necesidad
# # ---------------------------------------------------
YEAR_START = "2022-01-01T00:00:00"

# --------- Fecha de salida, se lleva el día actual ---------
# YEAR_END = "2024-12-31T00:00:00" # --- Si se requiere una fecha de salida especifica.
YEAR_END   = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

print("Rango de tiempo->", YEAR_START, "a", YEAR_END)

# --------- Dataset IDs (Socrata) ---------
DATASET_CONTRATOS       = "jbjy-vk9h"   # Contratos electrónicos
DATASET_MODIFICACIONES  = "u8cx-r425"   # Modificaciones
DATASET_ADICIONES       = "cb9c-h8sn"   # Adiciones
DATASET_SUSPENSIONES    = "u99c-7mfm"   # Suspensiones

BASE_URL = "https://www.datos.gov.co/resource"

# --------- Parámetros de API ---------
PAGE_SIZE   = 50000
ID_BATCH    = 80
PAUSE_S     = 0.25
MAX_RETRY   = 4

# Token de Socrata (opcional)
APP_TOKEN = (
    os.getenv("SODA_APP_TOKEN")
    or os.getenv("APP_TOKEN")
    or os.getenv("SOCRATA_APP_TOKEN")
)

def quitar_tildes(s):
    return (unicodedata.normalize("NFKD", s)
        .encode("ascii", "ignore")
        .decode("utf-8"))

def make_session(user_agent: str = "Colab/SECOP-Atenea") -> requests.Session:
    """Crea sesión HTTP con reintentos y App Token opcional."""
    s = requests.Session()
    s.headers.update({"User-Agent": user_agent})
    if APP_TOKEN:
        s.headers.update({"X-App-Token": APP_TOKEN})
    retry = Retry(
        total=MAX_RETRY,
        backoff_factor=0.7,
        status_forcelist=[429, 500, 502, 503, 504]
    )
    s.mount("https://", HTTPAdapter(max_retries=retry))
    return s

SESSION = make_session()

def soql_escape(value: str) -> str:
    """Escapa comillas simples para SoQL (dobla ' a '')."""
    if value is None:
        return ""
    return str(value).replace("'", "''").strip()

def soql_quote(value: str) -> str:
    """Devuelve el valor entre comillas simples escapando comillas internas."""
    if value is None:
        return "''"
    return "'" + soql_escape(value) + "'"

def quote_sql_list(values: List[Any]) -> str:
    """Convierte una lista en 'v1','v2','v3' para usar en IN (...)."""
    return ",".join(soql_quote(v) for v in values if v is not None)

def fetch_paginated(
    resource_id: str,
    where: Optional[str] = None,
    extra_params: Optional[Dict[str, Any]] = None,
    select: Optional[List[str]] = None, #### Selección de variables!!!
    session: Optional[requests.Session] = None,
    page_size: int = PAGE_SIZE,
) -> pd.DataFrame:
    """
    Descarga un recurso Socrata completo usando paginación $limit/$offset.
    """
    if session is None:
        session = SESSION

    url = f"{BASE_URL}/{resource_id}.json"
    all_records = []
    offset = 0

    while True:
        params = {
            "$limit": page_size,
            "$offset": offset,
        }
        if where:
            params["$where"] = where
        if extra_params:
            params.update(extra_params)

        attempt = 0
        while True:
            attempt += 1
            r = session.get(url, params=params, timeout=60)
            if r.status_code == 200:
                rows = r.json()
                if isinstance(rows, dict):
                    rows = [rows]
                break

            if r.status_code in (429, 500, 502, 503, 504) and attempt < MAX_RETRY:
                wait_s = min(10, 1.5 ** attempt)
                print(f"⏳ HTTP {r.status_code} offset={offset}; reintento {attempt}/{MAX_RETRY} en {wait_s:.1f}s")
                time.sleep(wait_s)
                continue

            raise requests.HTTPError(f"HTTP {r.status_code} - {r.text[:300]}\nURL: {r.url}")

        if not rows:
            break

        all_records.extend(rows)
        offset += page_size
        print(f"   +{len(rows)} filas (acum: {len(all_records)})")
        time.sleep(PAUSE_S)

    if not all_records:
        return pd.DataFrame()

    df = pd.DataFrame(all_records)
    return df

def fetch_for_ids(
    resource_id: str,
    id_list: List[Any],
    id_field: str = "id_contrato",
    page_size: int = PAGE_SIZE,
    id_batch_size: int = ID_BATCH,
    select: Optional[List[str]] = None,
    session: Optional[requests.Session] = None,
) -> pd.DataFrame:
    """
    Descarga un recurso Socrata filtrando por lotes de IDs:
        id_field IN ('id1','id2',...)
    con paginación $limit/$offset por lote.
    """
    if session is None:
        session = SESSION

    id_list = [str(x).strip() for x in list(id_list) if pd.notna(x)]
    if not id_list:
        print("La lista de ids está vacía. Nada que consultar.")
        return pd.DataFrame(columns=[id_field])

    total_ids = len(id_list)
    total_batches = math.ceil(total_ids / id_batch_size)
    print(f"Consultando recurso {resource_id} para {total_ids} contratos en {total_batches} lote(s)...")

    all_records = []
    url = f"{BASE_URL}/{resource_id}.json"

    for b in range(total_batches):
        batch = id_list[b * id_batch_size : (b + 1) * id_batch_size]
        if not batch:
            continue

        where = f"{id_field} IN ({quote_sql_list(batch)})"
        offset = 0

        while True:
            params = {
                "$select": ",".join(select) if select else "*",  #### Selección de variables!!!
                "$where": where,
                "$order": f"{id_field} ASC",
                "$limit": page_size,
                "$offset": offset,
            }

            attempt = 0
            while True:
                attempt += 1
                r = session.get(url, params=params, timeout=60)
                if r.status_code == 200:
                    try:
                        rows = r.json()
                    except Exception as e:
                        raise RuntimeError(
                            f"Respuesta no JSON. Código {r.status_code}. URL:\n{r.url}\n\n{r.text[:400]}"
                        ) from e
                    if isinstance(rows, dict):
                        rows = [rows]
                    break

                if r.status_code in (429, 500, 502, 503, 504) and attempt < MAX_RETRY:
                    wait_s = min(10, 1.5 ** attempt)
                    print(
                        f"⏳ {r.status_code} (lote {b+1}/{total_batches}, offset={offset}); "
                        f"reintento {attempt}/{MAX_RETRY} en {wait_s:.1f}s"
                    )
                    time.sleep(wait_s)
                    continue

                raise requests.HTTPError(f"HTTP {r.status_code} - {r.text[:300]}\nURL: {r.url}")

            if not rows:
                break

            all_records.extend(rows)
            offset += page_size
            print(f"   Lote {b+1}/{total_batches} +{len(rows)} (acum: {len(all_records)})")
            time.sleep(PAUSE_S)

        time.sleep(random.uniform(0.2, 0.6))

    if not all_records:
        print(f"No se recuperaron registros para {resource_id}.")
        return pd.DataFrame(columns=[id_field])

    df = pd.DataFrame(all_records).drop_duplicates()
    if id_field in df.columns:
        df[id_field] = df[id_field].astype(str).str.strip()
    return df

Rango de tiempo-> 2026-01-01T00:00:00 a 2026-02-25T18:35:00


# **Módulo 1: Extracción SECOP**

In [ ]:
print("\n=== MÓDULO 1: Extracción SECOP II ===")

# 1. Filtro SoQL para contratos
nombre_safe = soql_escape(ENTITY_NAME)
nit_safe    = soql_escape(ENTITY_NIT)

where_contratos = (
    f"fecha_de_firma between '{YEAR_START}' and '{YEAR_END}' "
    f"AND nombre_entidad = '{nombre_safe}' "
    f"AND nit_entidad = '{nit_safe}'"
)

print("Filtro SoQL Contratos:\n", where_contratos)

print("\nDescargando CONTRATOS ELECTRÓNICOS...")
df_contratos = fetch_paginated(
    resource_id=DATASET_CONTRATOS,
    where=where_contratos,
    extra_params={"$order": "fecha_de_firma ASC"},
    # select=CONTRATOS_SELECT, #### Selección de variables!!!
)

print(f"\nTotal contratos descargados: {len(df_contratos)}")
df_contratos.info()

if df_contratos.empty:
    raise RuntimeError("No se descargaron contratos. Revisa filtros de entidad/fechas.")

# Renombrar columnas añadiendo sufijo (Contratos_electronicos) excepto id_contrato
rename_map_contratos = {
    c: f"{c} (Contratos_electronicos)"
    for c in df_contratos.columns
    if c != "id_contrato" and "(Contratos_electronicos)" not in c
}
df_contratos = df_contratos.rename(columns=rename_map_contratos)

# Reordenar columnas
preferred_cols_base = [
    "nombre_entidad",
    "nit_entidad",
    "referencia_contrato",
    "objeto_contrato",
    "objeto_contractual",
    "proveedor_adjudicado",
    "documento_proveedor",
    "valor_del_contrato",
    "valor_total_adiciones",
    "fecha_de_firma",
    "fecha_inicio_ejecucion",
    "fecha_fin_ejecucion",
    "ciudad",
    "departamento",
    "sector",
    "link_secop_ii",
]

preferred_cols = [
    c for c in (preferred_cols_base + ["id_contrato"])
    if c in df_contratos.columns
]

other_cols = [c for c in df_contratos.columns if c not in preferred_cols]
df_contratos = df_contratos[preferred_cols + other_cols]

# Tipos amigables
for c in ["fecha_de_firma", "fecha_inicio_ejecucion", "fecha_fin_ejecucion"]:
    if c in df_contratos.columns:
        df_contratos[c] = pd.to_datetime(df_contratos[c], errors="coerce")

for c in ["valor_del_contrato", "valor_total_adiciones"]:
    if c in df_contratos.columns:
        df_contratos[c] = pd.to_numeric(df_contratos[c], errors="coerce")

SECOPII_ATENEA = df_contratos.copy()

ids = (
    SECOPII_ATENEA["id_contrato"]
    .dropna()
    .astype(str)
    .str.strip()
    .unique()
    .tolist()
)
print(f"\nid_contrato únicos en SECOPII_ATENEA: {len(ids):,}")

# MODIFICACIONES
print("\nDescargando MODIFICACIONES...")
df_modificaciones = fetch_for_ids(
    resource_id=DATASET_MODIFICACIONES,
    id_list=ids,
    id_field="id_contrato",
    # select=MODIFICACIONES_SELECT, #### Selección de variables!!!
)
print(f"\nRegistros de Modificaciones descargados: {len(df_modificaciones):,}")
df_modificaciones.info()

if not df_modificaciones.empty:
    mod_cols = [c for c in df_modificaciones.columns if c != "id_contrato"]
    df_modificaciones_ren = df_modificaciones.rename(
        columns={c: f"{c} (modificaciones)" for c in mod_cols}
    )
else:
    df_modificaciones_ren = df_modificaciones

SECOP_ATENEA_TOTAL = SECOPII_ATENEA.merge(
    df_modificaciones_ren,
    on="id_contrato",
    how="left",
)
print("\nMerge con MODIFICACIONES:")
print("   Filas ATENEA:", len(SECOPII_ATENEA))
print("   Filas Modificaciones:", len(df_modificaciones))
print("   Filas resultado (LEFT):", len(SECOP_ATENEA_TOTAL))

# ADICIONES
print("\nDescargando ADICIONES...")
df_adiciones = fetch_for_ids(
    resource_id=DATASET_ADICIONES,
    id_list=ids,
    id_field="id_contrato",
    # select=MODIFICACIONES_SELECT, #### Selección de variables!!!
)
print(f"\nRegistros de Adiciones descargados: {len(df_adiciones):,}")
df_adiciones.info()

if not df_adiciones.empty:
    adic_cols = [c for c in df_adiciones.columns if c != "id_contrato"]
    df_adiciones_ren = df_adiciones.rename(
        columns={c: f"{c} (adiciones)" for c in adic_cols}
    )
else:
    df_adiciones_ren = df_adiciones

SECOP_ATENEA_TOTAL = SECOP_ATENEA_TOTAL.merge(
    df_adiciones_ren,
    on="id_contrato",
    how="left",
)
print("\nMerge con ADICIONES:")
print("   Filas Adiciones:", len(df_adiciones))
print("   Filas resultado (LEFT):", len(SECOP_ATENEA_TOTAL))

# SUSPENSIONES
print("\nDescargando SUSPENSIONES...")
df_suspensiones = fetch_for_ids(
    resource_id=DATASET_SUSPENSIONES,
    id_list=ids,
    id_field="id_contrato",
    # select=MODIFICACIONES_SELECT, #### Selección de variables!!!
)
print(f"\nRegistros de Suspensiones descargados: {len(df_suspensiones):,}")
df_suspensiones.info()

if not df_suspensiones.empty:
    susp_cols = [c for c in df_suspensiones.columns if c != "id_contrato"]
    df_suspensiones_ren = df_suspensiones.rename(
        columns={c: f"{c} (suspensiones)" for c in susp_cols}
    )
else:
    df_suspensiones_ren = df_suspensiones

SECOP_ATENEA_TOTAL = SECOP_ATENEA_TOTAL.merge(
    df_suspensiones_ren,
    on="id_contrato",
    how="left",
)
print("\nMerge con SUSPENSIONES:")
print("   Filas Suspensiones:", len(df_suspensiones))
print("   Filas resultado (LEFT):", len(SECOP_ATENEA_TOTAL))

print("   SECOP_ATENEA_TOTAL:", SECOP_ATENEA_TOTAL.shape)


=== MÓDULO 1: Extracción SECOP II ===
Filtro SoQL Contratos:
 fecha_de_firma between '2026-01-01T00:00:00' and '2026-02-25T18:35:00' AND nombre_entidad = 'AGENCIA DISTRITAL PARA LA EDUCACIÓN SUPERIOR, LA CIENCIA Y LA TECNOLOGÍA, ATENEA' AND nit_entidad = '901508361'

Descargando CONTRATOS ELECTRÓNICOS...
   +417 filas (acum: 417)

Total contratos descargados: 417
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417 entries, 0 to 416
Data columns (total 86 columns):
 #   Column                                                            Non-Null Count  Dtype 
---  ------                                                            --------------  ----- 
 0   nombre_entidad                                                    417 non-null    object
 1   nit_entidad                                                       417 non-null    object
 2   departamento                                                      417 non-null    object
 3   ciudad                                               

# **Módulo 2: PAA + Dependencias**

In [ ]:
print("\n=== MÓDULO 2: PAA y Dependencias ===")

# Dataset PAA
PAA_RESOURCE_ID = "9sue-ezhx"
PAA_BASE_URL = f"https://www.datos.gov.co/resource/{PAA_RESOURCE_ID}.json"
ENTIDAD_PAA = "AGENCIA DISTRITAL PARA LA EDUCACIÓN SUPERIOR LA CIENCIA Y LA TECNOLOGÍA ATENEA"
UMBRAL = 85  # fuzzy

# Descargar PAA Detalles
params = {
    "$where": f"nombre_entidad='{ENTIDAD_PAA}'",
    "$limit": 50000,
}
print("\nDescargando PAA Detalles para la entidad…")
r = requests.get(PAA_BASE_URL, params=params, timeout=60)
r.raise_for_status()
data = r.json()
PAA_DETALLES_ATENEA = pd.DataFrame(data)
print(f"Registros PAA descargados: {len(PAA_DETALLES_ATENEA)}")
PAA_DETALLES_ATENEA.info()

if PAA_DETALLES_ATENEA.empty:
    raise RuntimeError("No se descargaron registros del PAA para la entidad configurada.")

# Crear ID_PAA_SECOP en PAA
PAA_DETALLES_ATENEA["ID_PAA_SECOP"] = (
    PAA_DETALLES_ATENEA["descripcion"].astype(str).fillna("").str.strip()
    + " "
    + PAA_DETALLES_ATENEA["procesos_relacionados"].astype(str).fillna("").str.strip()
)
print("\nEjemplos PAA ID_PAA_SECOP:")
print(PAA_DETALLES_ATENEA[["descripcion", "procesos_relacionados", "ID_PAA_SECOP"]].head())

# Crear ID_PAA_SECOP en SECOP
col_desc_secop = "descripcion_del_proceso (Contratos_electronicos)"
col_ref_secop  = "referencia_del_contrato (Contratos_electronicos)"

for c in [col_desc_secop, col_ref_secop]:
    if c not in SECOP_ATENEA_TOTAL.columns:
        raise KeyError(f"No encuentro la columna '{c}' en SECOP_ATENEA_TOTAL.")

SECOP_ATENEA_TOTAL["ID_PAA_SECOP"] = (
    SECOP_ATENEA_TOTAL[col_desc_secop].astype(str).fillna("").str.strip()
    + " "
    + SECOP_ATENEA_TOTAL[col_ref_secop].astype(str).fillna("").str.strip()
)

# Fuzzy matching
def _norm_txt(s: object) -> str:
    """Normaliza: minúsculas, sin tildes, solo [a-z0-9], colapsa espacios."""
    if pd.isna(s):
        return ""
    s = str(s).strip().lower()
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

SECOP_ATENEA_TOTAL.columns = SECOP_ATENEA_TOTAL.columns.str.strip()
PAA_DETALLES_ATENEA.columns = PAA_DETALLES_ATENEA.columns.str.strip()

left = SECOP_ATENEA_TOTAL.copy()
right = PAA_DETALLES_ATENEA.copy()

left["_key_norm"] = left["ID_PAA_SECOP"].map(_norm_txt)
right["_key_norm"] = right["ID_PAA_SECOP"].map(_norm_txt)

choices = right["_key_norm"].astype(str).tolist()

best_idx: List[Optional[int]] = []
best_scr: List[Optional[float]] = []

print("\n Ejecutando fuzzy matching PAA ↔ SECOP…")
for s in tqdm(left["_key_norm"].astype(str), total=len(left), desc="Fuzzy matching"):
    if not s:
        best_idx.append(np.nan)
        best_scr.append(np.nan)
        continue

    m = process.extractOne(s, choices, scorer=fuzz.token_set_ratio, score_cutoff=0)
    if not m:
        best_idx.append(np.nan)
        best_scr.append(np.nan)
    else:
        _, score, idx = m
        if score >= UMBRAL:
            best_idx.append(int(idx))
            best_scr.append(float(score))
        else:
            best_idx.append(np.nan)
            best_scr.append(float(score))

left["PAA_matchingScore"] = pd.Series(best_scr, dtype="float").round(2)
print("\nResumen PAA_matchingScore:")
print(left["PAA_matchingScore"].describe())

# Left join SECOP <- PAA (prefijo PAA::)
cols_right = [c for c in right.columns if c != "_key_norm"]
paa_take = pd.DataFrame(index=left.index)
mapped = pd.Series(best_idx, index=left.index, dtype="float")
valid = mapped.dropna().astype(int)

if len(valid) > 0:
    right_sel = right.iloc[valid.values][cols_right].copy()
    right_sel.index = valid.index
    for c in cols_right:
        paa_take[f"PAA::{c}"] = right_sel.reindex(left.index)[c]

merged = pd.concat(
    [left.drop(columns=["_key_norm"], errors="ignore"), paa_take],
    axis=1,
)
SECOP_ATENEA_TOTAL = merged
print("\n SECOP_ATENEA_TOTAL tras join con PAA:")
SECOP_ATENEA_TOTAL.info()
print("   SECOP_ATENEA_TOTAL:", SECOP_ATENEA_TOTAL.shape)


=== MÓDULO 2: PAA y Dependencias ===

Descargando PAA Detalles para la entidad…
Registros PAA descargados: 18716
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18716 entries, 0 to 18715
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   id                              18716 non-null  object
 1   identificador_unico             18716 non-null  object
 2   descripcion                     18716 non-null  object
 3   fecha_esperada_de_inicio        18716 non-null  object
 4   fecha_esperada_de_recepcion     18716 non-null  object
 5   duracion_esperada               18716 non-null  object
 6   unidad_de_duracion_esperada     18716 non-null  object
 7   origen_recursos                 18716 non-null  object
 8   valor_total_esperado            18716 non-null  object
 9   valor_esperado_de_presupuesto   18716 non-null  object
 10  requiere_vigencias_futuras      18716 non-null  obje

Fuzzy matching:   0%|          | 0/449 [00:00<?, ?it/s]


Resumen PAA_matchingScore:
count    449.000000
mean      98.685791
std        1.318197
min       93.750000
25%       97.770000
50%       98.720000
75%      100.000000
max      100.000000
Name: PAA_matchingScore, dtype: float64

 SECOP_ATENEA_TOTAL tras join con PAA:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Columns: 159 entries, id_contrato to PAA::ID_PAA_SECOP
dtypes: bool(1), float64(1), object(157)
memory usage: 554.8+ KB
   SECOP_ATENEA_TOTAL: (449, 159)


# **Módulo 3: Transformaciones y variables inferidas**

In [ ]:
print("\n=== MÓDULO 3: Transformaciones y Variables Inferidas ===")

# 1. AÑO_INFERIDO
col_ref = "referencia_del_contrato (Contratos_electronicos)"
if col_ref not in SECOP_ATENEA_TOTAL.columns:
    raise KeyError(f"No existe la columna '{col_ref}' en la base SECOP_ATENEA_TOTAL.")

#- Limpiar Referencia del contrato
s = SECOP_ATENEA_TOTAL[col_ref].astype("string")
s = (s.str.upper()
     .str.replace(r"\bATENA\b", "ATENEA", regex=True)
     .str.replace(".", "", regex=False)
     .str.replace(",", "", regex=False)
     .str.replace(r"\s+", " ", regex=True)
     .str.strip()
     .str.replace(r"-+$", "", regex=True)) # Quitar guión al final si existe
# 1) Si viene por ejemplo "ATENEA 243-2025" o "ATENEA 067-2024" => "ATENEA-243-2025"
s = s.str.replace(r"^ATENEA\s+(?=\d)", "ATENEA-", regex=True)
# 2) Normaliza guiones (espacios alrededor) => "243 - 2025" -> "243-2025"
s = s.str.replace(r"\s*-\s*", "-", regex=True)
# 3) Si viene por ejemplo "349-2026" sin prefijo => "ATENEA-349-2026"
s = s.str.replace(r"^(?=\d{1,4}-\d{4}$)", "ATENEA-", regex=True)
# 4) Casos particulares (manuales)
s = s.replace({
    "ATENEA-097": "ATENEA-097-2022",   # ATENEA 097 (sin año) -> año real 2022
    "ATENEA 097": "ATENEA-097-2022",   # por si quedó con espacio antes del paso 1
    "ATENEA097":  "ATENEA-097-2022",   # por si llega pegado
    "ATENEA-319-202": "ATENEA-319-2023" # Nuevo caso
})
SECOP_ATENEA_TOTAL[col_ref] = s

### --- Inferir año ---
ref = SECOP_ATENEA_TOTAL[col_ref]
anio = np.where(
    ref.str.startswith("CO1", na=False), 2022,
    pd.to_numeric(ref.str.extract(r"(\d{4})\s*$")[0], errors="coerce"))

SECOP_ATENEA_TOTAL["AÑO_INFERIDO"] = pd.Series(anio, index=SECOP_ATENEA_TOTAL.index).astype("Int64")


# 2. TIPO_PERSONA_INFERIDO
# -- Cambio específico.
SECOP_ATENEA_TOTAL.loc[
    SECOP_ATENEA_TOTAL["documento_proveedor (Contratos_electronicos)"].isin(['79574594', '80777962']),
    "tipodocproveedor (Contratos_electronicos)"
] = 'Cédula de Ciudadanía'

col_doc = "tipodocproveedor (Contratos_electronicos)"
col_nom = "proveedor_adjudicado (Contratos_electronicos)"

for c in [col_doc, col_nom]:
    if c not in SECOP_ATENEA_TOTAL.columns:
        raise KeyError(f"No existe la columna '{c}' necesaria para TIPO_PERSONA_INFERIDO.")

# Normalización vectorizada (en vez de apply)
doc = (SECOP_ATENEA_TOTAL[col_doc].astype("string").fillna("")
       .str.upper().str.strip()
       .str.normalize("NFKD")                     # descompone tildes
       .str.encode("ascii", errors="ignore")      # quita tildes
       .str.decode("utf-8"))

nom = (SECOP_ATENEA_TOTAL[col_nom].astype("string").fillna("")
       .str.upper().str.strip()
       .str.normalize("NFKD")
       .str.encode("ascii", errors="ignore")
       .str.decode("utf-8"))

# Patrones jurídica (regex compilado)
PATRONES_JURIDICA = r"\b(SAS|S\.A|S\. A|LTDA|ASOCIACION|UNIVERSIDAD|EMPRESA|COOPERATIVA|CORPORACION|CONSORCIO|FUNDACION)\b"

is_cedula = doc.str.contains("CEDULA", na=False)
is_ppt    = doc.str.contains("PERMISO POR PROTECCION TEMPORAL", na=False)
is_nit    = doc.str.contains("NIT", na=False)
is_jur_by_name = nom.str.contains(PATRONES_JURIDICA, regex=True, na=False)

SECOP_ATENEA_TOTAL["TIPO_PERSONA_INFERIDO"] = pd.NA
SECOP_ATENEA_TOTAL.loc[is_cedula | is_ppt, "TIPO_PERSONA_INFERIDO"] = "Natural"
SECOP_ATENEA_TOTAL.loc[~(is_cedula | is_ppt) & (is_nit | is_jur_by_name), "TIPO_PERSONA_INFERIDO"] = "Jurídica"

# 3. SUSPENSIÓN_INFERIDO
col_susp = "tipo (suspensiones)"
if col_susp in SECOP_ATENEA_TOTAL.columns:
    cond = (
        SECOP_ATENEA_TOTAL[col_susp]
        .astype("string")
        .fillna("")              # <- clave
        .str.strip()
        .str.upper()
        .eq("SUSPENSION")
    )

    SECOP_ATENEA_TOTAL["SUSPENSIÓN_INFERIDO"] = np.where(cond.to_numpy(), "SI", "NO")
else:
    SECOP_ATENEA_TOTAL["SUSPENSIÓN_INFERIDO"] = pd.NA

# 4. PLAZOS DE EJECUCIÓN
col_inicio_dt = "fecha_de_inicio_del_contrato (Contratos_electronicos)"
col_fin_dt   = "fecha_de_fin_del_contrato (Contratos_electronicos)"

for c in [col_inicio_dt, col_fin_dt]:
    if c not in SECOP_ATENEA_TOTAL.columns:
        raise KeyError(f"No existe la columna '{c}' necesaria para plazos de ejecución.")

inicio = pd.to_datetime(SECOP_ATENEA_TOTAL[col_inicio_dt], errors="coerce")
fin   = pd.to_datetime(SECOP_ATENEA_TOTAL[col_fin_dt], errors="coerce")

# DÍAS
plazo_dias = (fin - inicio).dt.days
# MESES (aprox como tu days/30)
plazo_meses = (plazo_dias / 30).round(1)

SECOP_ATENEA_TOTAL["PLAZO DE EJECUCIÓN FINAL DEL CONTRATO (DÍAS)_inferido"] = plazo_dias
SECOP_ATENEA_TOTAL["PLAZO DE EJECUCIÓN EN MESES (INICIAL)_inferido"] = plazo_meses

# 5.  porc_avanceplazo_inferido
hoy = pd.Timestamp(datetime.now().date())

# Fecha de corte = hoy si aún no termina, si no fecha fin
fecha_corte = pd.concat([fin, pd.Series(hoy, index=fin.index)], axis=1).min(axis=1)
dias_trans = (fecha_corte - inicio).dt.days.clip(lower=0)

avance = (dias_trans / plazo_dias) * 100
avance = avance.where(plazo_dias > 0)
SECOP_ATENEA_TOTAL["porc_avanceplazo_inferido"] = avance.clip(0, 100)


=== MÓDULO 3: Transformaciones y Variables Inferidas ===


/tmp/ipython-input-4171594842.py:74: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  is_jur_by_name = nom.str.contains(PATRONES_JURIDICA, regex=True, na=False)


In [ ]:
# 6. ESTADO VIGENTE_inferido
# ---- fecha fin ----
fecha_fin = pd.to_datetime(
    SECOP_ATENEA_TOTAL["fecha_de_fin_del_contrato (Contratos_electronicos)"],
    errors="coerce",
    format="%Y-%m-%dT%H:%M:%S.%f")

hoy = pd.Timestamp.today().normalize()

# ---- normalizar estado contrato ----
estado_norm = (
    SECOP_ATENEA_TOTAL["estado_contrato (Contratos_electronicos)"]
    .astype("string").str.lower().str.strip())

# ---- condición adicional terminado/cerrado ----
cond_no_estado = estado_norm.isin(["terminado", "cerrado"])

# ---- variable final ----
SECOP_ATENEA_TOTAL["ESTADO VIGENTE_inferido"] = pd.Series(
    np.where(cond_no_estado, "NO",
        np.where(fecha_fin > hoy, "SI", "NO")
    ), index=SECOP_ATENEA_TOTAL.index, dtype="string"
).mask(fecha_fin.isna(), pd.NA)


# 7. TERMINACIÓN ANTICIPADA_inferido

COL_MOD = "proposito_modificacion (modificaciones)"
COL_ADI = "descripcion (adiciones)"

# Regex (el mismo concepto, puedes conservar tu patrón si quieres)
PATRON_TERMINACION_ANT = (
    r"\b("
    r"(dar\s+por\s+terminad\w+)\s+(de\s+manera\s+)?anticipad\w*"
    r"|termin(ar|ad[oa]?|acion)\s+(de\s+manera\s+)?anticipad\w*"
    r")\b")

def norm_serie(ser: pd.Series) -> pd.Series:
    """Normaliza vectorizado: lower, strip, quitar tildes, colapsar espacios."""
    ser = ser.astype("string").fillna("").str.lower().str.strip()
    # Quitar tildes (vectorizado)
    ser = (ser.str.normalize("NFKD")
              .str.encode("ascii", errors="ignore")
              .str.decode("utf-8"))
    # Colapsar espacios
    ser = ser.str.replace(r"\s+", " ", regex=True)
    return ser

mask_mod = pd.Series(False, index=SECOP_ATENEA_TOTAL.index)
if COL_MOD in SECOP_ATENEA_TOTAL.columns:
    s_mod = norm_serie(SECOP_ATENEA_TOTAL[COL_MOD])
    mask_mod = s_mod.str.contains(PATRON_TERMINACION_ANT, regex=True, na=False)

mask_adi = pd.Series(False, index=SECOP_ATENEA_TOTAL.index)
if COL_ADI in SECOP_ATENEA_TOTAL.columns:
    s_adi = norm_serie(SECOP_ATENEA_TOTAL[COL_ADI])
    mask_adi = s_adi.str.contains(PATRON_TERMINACION_ANT, regex=True, na=False)

mask_total = mask_mod | mask_adi

SECOP_ATENEA_TOTAL["TERMINACIÓN ANTICIPADA_inferido"] = np.where(mask_total, "SI", "NO")

# 8. OBSERVACIONES_inferido
col_mod = COL_MOD
col_adi = COL_ADI

def combinar_observaciones(mod, adi):
    mod = str(mod).strip() if pd.notna(mod) else ""
    adi = str(adi).strip() if pd.notna(adi) else ""

    if mod and adi:
        return f"{mod} | {adi}"
    elif mod:
        return mod
    elif adi:
        return adi
    else:
        return np.nan

SECOP_ATENEA_TOTAL["OBSERVACIONES_inferido"] = SECOP_ATENEA_TOTAL.apply(
    lambda r: combinar_observaciones(r.get(col_mod), r.get(col_adi)),
    axis=1,
)

# 9. CESIÓN_inferido
col_estado = "estado_contrato (Contratos_electronicos)"
col_tipo_adi = "tipo (adiciones)"
col_proposito_mod = COL_MOD
col_desc_adi = COL_ADI

def normalizar_texto(s):
    if pd.isna(s):
        return ""
    s = str(s).lower().strip()
    s = unicodedata.normalize("NFD", s)
    s = "".join(ch for ch in s if unicodedata.category(ch) != "Mn")
    return s

def es_cesion(row):
    estado = normalizar_texto(row.get(col_estado, ""))
    tipo_adi = normalizar_texto(row.get(col_tipo_adi, ""))
    proposito = normalizar_texto(row.get(col_proposito_mod, ""))
    descripcion = normalizar_texto(row.get(col_desc_adi, ""))

    cond1 = estado == "cedido"
    cond2 = tipo_adi == "cesion"
    cond3 = bool(re.search(r"\bcesion\b", proposito))
    cond4 = bool(re.search(r"\bcesion\b", descripcion))

    return "SI" if (cond1 or cond2 or cond3 or cond4) else "NO"

SECOP_ATENEA_TOTAL["CESIÓN_inferido"] = SECOP_ATENEA_TOTAL.apply(es_cesion, axis=1)

# 10. Fórmula DIAN.
def limpiar_nit_base(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    if "-" in s:
        s = s.split("-", 1)[0]
    s = re.sub(r"\D", "", s)
    return np.nan if s == "" else s

WEIGHTS = [3, 7, 13, 17, 19, 23, 29, 37, 41, 43, 47, 53, 59, 67, 71]
def dv_dian(nit_base):
    if pd.isna(nit_base):
        return np.nan
    s = str(nit_base)
    if len(s) > len(WEIGHTS):
        s = s[-len(WEIGHTS):]
    total = 0
    for i, d in enumerate(reversed(s)):
        total += int(d) * WEIGHTS[i]
    r = total % 11
    return r if r in (0, 1) else 11 - r

# 10.1. Digito de verificación para proveedor
COL_NIT = "documento_proveedor (Contratos_electronicos)"
if COL_NIT not in SECOP_ATENEA_TOTAL.columns:
    raise KeyError(f"No existe la columna '{COL_NIT}' necesaria para el cálculo DIAN.")

SECOP_ATENEA_TOTAL["nit_base"] = SECOP_ATENEA_TOTAL[COL_NIT].apply(limpiar_nit_base)
SECOP_ATENEA_TOTAL["DígitoVerificaciónDIAN_DV"] = (
    SECOP_ATENEA_TOTAL["nit_base"].apply(dv_dian).astype("Int64"))

# 10.2. Digito de verificación para supervisor
COL_NIT2 = "n_mero_de_documento_supervisor (Contratos_electronicos)"
if COL_NIT2 not in SECOP_ATENEA_TOTAL.columns:
    raise KeyError(f"No existe la columna '{COL_NIT2}' necesaria para el cálculo DIAN.")

SECOP_ATENEA_TOTAL["nit_base_supervisor"] = SECOP_ATENEA_TOTAL[COL_NIT2].apply(limpiar_nit_base)
SECOP_ATENEA_TOTAL["DígitoVerificaciónDIAN_DV_supervisor"] = (
    SECOP_ATENEA_TOTAL["nit_base_supervisor"].apply(dv_dian).astype("Int64"))

# 11. Indicadores financieros básicos
valor_contrato = "valor_del_contrato (Contratos_electronicos)"
valor_pagado   = "valor_pagado (Contratos_electronicos)"

SECOP_ATENEA_TOTAL[valor_contrato] = (
    pd.to_numeric(SECOP_ATENEA_TOTAL[valor_contrato], errors="coerce")
    .fillna(0))

SECOP_ATENEA_TOTAL[valor_pagado] = (
    pd.to_numeric(SECOP_ATENEA_TOTAL[valor_pagado], errors="coerce")
    .fillna(0))

SECOP_ATENEA_TOTAL["porc_ejecucion_financiera"] = np.where(
    SECOP_ATENEA_TOTAL[valor_contrato] > 0,
    (SECOP_ATENEA_TOTAL[valor_pagado] / SECOP_ATENEA_TOTAL[valor_contrato]) * 100,
    np.nan)


   SECOP_ATENEA_TOTAL: (449, 174)


/tmp/ipython-input-2799151536.py:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_mod = s_mod.str.contains(PATRON_TERMINACION_ANT, regex=True, na=False)
/tmp/ipython-input-2799151536.py:56: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_adi = s_adi.str.contains(PATRON_TERMINACION_ANT, regex=True, na=False)


In [ ]:
# 12. valor_pago_mensual_inferido
SECOP_ATENEA_TOTAL["valor_pago_mensual_inferido"] =  SECOP_ATENEA_TOTAL["valor_del_contrato (Contratos_electronicos)"]/SECOP_ATENEA_TOTAL["PLAZO DE EJECUCIÓN EN MESES (INICIAL)_inferido"].round(1)

# 13. Tipo de modificación
# Lógica:
# - tipo (adiciones)  → fuente principal
# - tipo (suspensiones) → solo respaldo

# Normalizar primero ambas columnas
adiciones = SECOP_ATENEA_TOTAL["tipo (adiciones)"].str.strip().str.lower()
suspensiones = SECOP_ATENEA_TOTAL["tipo (suspensiones)"].str.strip().str.lower()

# Crear la nueva variable
SECOP_ATENEA_TOTAL["tipo_modificacion"] = adiciones

# Condición: cuando adiciones sea "no definido" o NaN
condicion = adiciones.isna() | (adiciones == "no definido")

# Reemplazar solo si suspensiones NO es NaN
SECOP_ATENEA_TOTAL["tipo_modificacion"] = (
    SECOP_ATENEA_TOTAL["tipo_modificacion"]
        .mask(condicion & suspensiones.notna(), suspensiones)
        .str.capitalize())

# 14. Prorroga inferido
SECOP_ATENEA_TOTAL["prorroga_inferido"] = np.where(
    SECOP_ATENEA_TOTAL["dias_adicionados (contratos_electronicos)"].
    fillna(0) > 0, "SI", "NO")

# 15. NATURALEZA_INFERIDO
# Privada o Pública

# Palabras clave entidades públicas
patrones_publicos = [
    r"\bSECRETARIA\b",
    r"\bMINISTERIO\b",
    r"\bALCALDIA\b",
    r"\bGOBERNACION\b",
    r"\bDISTRITAL\b",
    r"\bNACIONAL\b",
    r"\bESE\b",
    r"\bSUBRED\b",
    r"\bSENA\b",
    r"\bAGENCIA\b",
    r"\bSUPERINTENDENCIA\b",
    r"\bCOMISION\b",
    r"\bFONDO\b",
    r"\bTRANSMILENIO\b",
    r"\bCANAL CAPITAL\b",
    r"\bETB\b",
    r"\bSERVICIOS POSTALES NACIONALES\b",
    r"\bGRUPO ENERGIA\b",
    r"\bARMADA\b",
    r"\bNAVAL\b",
    r"\bEJERCITO\b",
    r"\bPOLICIA\b",
    r"\bFUERZA\b",
    r"\bJEFATURA\b",
    r"\bICETEX\b",
    r"\bCREDITO EDUCATIVO\b",
    r"\bINSTITUTO\b",
    r"\bUNIDAD ADMINISTRATIVA\b",
    r"\bESTABLECIMIENTO PUBLICO\b",
    r"\bEMPRESA SOCIAL DEL ESTADO\b",
    r"\bEMPRESA INDUSTRIAL\b",
    r"\bEICE\b",
    r"\bCONTRALORIA\b",
    r"\bPROCURADURIA\b",
    r"\bDEFENSORIA\b",
    r"\bREGISTRADURIA\b",
    r"\bTRIBUNAL\b",
    r"\bJUZGADO\b",
    r"\bCORTE\b",
    r"\bFISCALIA\b"
]

# Palabras clave Uuniversidades públicas
universidades_publicas_colombia = [
    "Universidad Nacional de Colombia",
    "Universidad de Antioquia",
    "Universidad del Valle",
    "Universidad Industrial de Santander",
    "Universidad de Cartagena",
    "Universidad del Atlántico",
    "Universidad del Cauca",
    "Universidad de Caldas",
    "Universidad de Córdoba",
    "Universidad de Cundinamarca",
    "Universidad de la Amazonía",
    "Universidad de La Guajira",
    "Universidad del Magdalena",
    "Universidad del Pacífico",
    "Universidad del Quindío",
    "Universidad del Tolima",
    "Universidad Popular del Cesar",
    "Universidad Pedagógica Nacional",
    "Universidad Pedagógica y Tecnológica de Colombia",
    "Universidad Surcolombiana",
    "Universidad de los Llanos",
    "Universidad Distrital Francisco José de Caldas",
    "Universidad Tecnológica de Pereira",
    "Universidad de Nariño",
    "Universidad de Pamplona",
    "Universidad Francisco de Paula Santander",
    "Universidad Francisco de Paula Santander - Ocaña",
    "Universidad Colegio Mayor de Cundinamarca",
    "Universidad Tecnológica del Chocó Diego Luis Córdoba",
    "Universidad de Sucre",
    "Universidad Militar Nueva Granada",
    "Universidad Nacional Abierta y a Distancia",
    "Universidad Autónoma Indígena Intercultural",
    "UNAD",
    "UPTC"
]

# 1. Compilar regex de patrones públicos
regex_publicos = re.compile("|".join(patrones_publicos), flags=re.IGNORECASE)

# 2. Preparar universidades en mayúsculas
unis_publicas_set = {u.strip().upper() for u in universidades_publicas_colombia if isinstance(u, str) and u.strip()}

col_tipo = "TIPO_PERSONA_INFERIDO"
col_prov = "proveedor_adjudicado (Contratos_electronicos)"

# Normalizar columnas
tipo_upper = SECOP_ATENEA_TOTAL[col_tipo].fillna("").astype(str).str.upper()
proveedor_norm = SECOP_ATENEA_TOTAL[col_prov].astype("string").fillna("").apply(quitar_tildes)

# --- Banderas de "Pública" por nombre ---
# a) por patrones regex
mask_regex_publico = proveedor_norm.str.contains(regex_publicos, na=False)

# b) por universidades (substring en mayúsculas)
proveedor_upper = proveedor_norm.str.upper()
mask_uni_publica = proveedor_upper.apply(lambda x: any(u in x for u in unis_publicas_set))

mask_nombre_publico = mask_regex_publico | mask_uni_publica

# --- Regla final ---
mask_juridica = tipo_upper.eq("JURÍDICA")

SECOP_ATENEA_TOTAL["NATURALEZA_INFERIDO"] = np.where(
    mask_juridica & mask_nombre_publico,
    "Pública",
    "Privada")

# 16. CLASIFICACIÓN_INFERIDO
# Persona Natural, Privadas, Universidades, Públicos

# Palabras clave universidades
patrones_universidades = [

    # Universidades directas
    r"\bUNIVERSIDAD\b",

    # Tipologías IES
    r"\bUNIVERSITARIA\b",
    r"\bESCUELA\b",
    r"\bPOLITECNICO\b",
    r"\bCOLEGIO\b",
    r"\bINSTITUTO TECNICO\b",
    r"\bACADEMIA\b",
    r"\bTECNOLOGICA\b",
    r"\bEDUCACION SUPERIOR\b",

    # Siglas comunes IES Colombia
    r"\bCUN\b",
    r"\bUNAD\b",
    r"\bUPTC\b",
    r"\bUDCA\b",
    r"\bUAN\b",
    r"\bINCAP\b",
    r"\bCESDE\b",
    r"\bTEINCO\b",
    r"\bCIDE\b",
    r"\bECCI\b",
    r"\bEAN\b",
    r"\bUNISALLE\b",
    r"\bUNIMINUTO\b",
    r"\bUNIAGRARIA\b",
    r"\bFUCS\b",

    # Institutos específicos relevantes
    r"\bKONRAD LORENZ\b"
]

# Compilar patrones universidades
regex_universidades = re.compile("|".join(patrones_universidades), flags=re.IGNORECASE)

# Detectar universidades por nombre del proveedor
mask_universidad = proveedor_norm.str.contains(regex_universidades, na=False)

# Detectar públicos/privados usando la variable ya creada
mask_publico = SECOP_ATENEA_TOTAL["NATURALEZA_INFERIDO"].eq("Pública")

SECOP_ATENEA_TOTAL["CLASIFICACIÓN_INFERIDO"] = np.select(
    [
        ~mask_juridica,                         # no jurídica
        mask_juridica & mask_universidad,       # jurídica y universidad
        mask_juridica & ~mask_universidad & mask_publico,  # jurídica, no universidad, pública
        mask_juridica & ~mask_universidad & ~mask_publico  # jurídica, no universidad, privada
    ],
    ["Persona Natural",
        "Universidades",
        "Públicos",
        "Privadas"],
    default="Privadas"  # fallback seguro
)

In [ ]:
############################################################
###======= Componentes PAA::ID_PAA_SECOP (<2026) ========###
############################################################

# ================================================
# 1) DICCIONARIO DEPENDENCIAS
# ================================================
Diccionario_Dependencia = pd.DataFrame(
    [
        ("DIRECCIÓN GENERAL", "DES"),
        ("GERENCIA DE CIENCIA, TECNOLOGÍA E INNOVACIÓN", "GCTI"),
        ("GERENCIA DE CIENCIA, TECNOLOGÍA E INNOVACIÓN", "CTEI"),
        ("GERENCIA DE EDUCACIÓN POSMEDIA", "GEP"),
        ("GERENCIA DE ESTRATEGIA", "GE"),
        ("OFICINA JURÍDICA", "OJ"),
        ("SUB. GESTIÓN_ADMINISTRATIVA", "SGA"),
        ("SUBGERENCIA DE ANÁLISIS DE INFORMACIÓN Y GESTIÓN DEL CONOCIMIENTO", "SAIGC"),
        ("SUBGERENCIA DE PLANEACIÓN", "SPL"),
        ("SUBGERENCIA DE TECNOLOGÍAS DE LA INFORMACIÓN Y LAS COMUNICACIONES", "TIC"),
        ("SUBGERENCIA FINANCIERA", "SGF"),
        ("OFICINA CONTROL INTERNO DISCIPLINARIO", "CID"),
        ("COMUNICACIONES", "COM"),
        ("GERENCIA DE GESTIÓN CORPORATIVA", "GGC"),
        ("OFICINA CONTROL INTERNO DISCIPLINARIO", "OCID"),
        ("OFICINA DE CONTROL INTERNO DE GESTIÓN", "OCIG"),
        ("TESORERIA", "TES"),
    ],
    columns=["DEPENDENCIA", "CODIGO_ID"],
)

# Mapa código -> dependencia
mapa_dep = dict(zip(Diccionario_Dependencia["CODIGO_ID"], Diccionario_Dependencia["DEPENDENCIA"]))

# ================================================
# 2) ALIAS / CORRECCIONES PUNTUALES
# ================================================
alias_codigos = {
    "SF": "SGF",                 # SF es SGF
    "OCDI": "OCID",              # OCDI es OCID

    # Variantes textuales de PLANEACIÓN
    "SUB-PLANEACION": "SPL",
    "SUBPLANEACION": "SPL",
    "SUB_PLANEACION": "SPL",

    # Opcional: si todo lo que diga SUBG-PLANEAC debe caer en SPL
    "SUBG-PLANEAC": "SPL",
    "SUBGPLANEAC": "SPL",
    "SUBG_PLANEAC": "SPL",
    "SPL": "SPL"
}

# ================================================
# 3) IDENTIFICAR LA COLUMNA PAA::descripcion
# ================================================
col_desc_paa = "PAA::descripcion"
if col_desc_paa not in SECOP_ATENEA_TOTAL.columns:
    lookup = {c.strip().lower(): c for c in SECOP_ATENEA_TOTAL.columns}
    if "paa::descripcion" in lookup:
        col_desc_paa = lookup["paa::descripcion"]
    else:
        raise KeyError("No encuentro la columna 'PAA::descripcion' en SECOP_ATENEA_TOTAL.")

# ================================================
# 4) LIMPIEZA PREVIA DEL TEXTO
# ================================================
# - Reemplaza NBSP (\u00A0) por espacio normal
# - Colapsa espacios múltiples
# - Strip final
desc = (
    SECOP_ATENEA_TOTAL[col_desc_paa]
    .astype(str)
    .str.replace("\u00A0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip())

# ================================================
# 5) EXTRACCIÓN ROBUSTA (2 PASOS)
#    Paso A: casos especiales (SUB-PLANEACION pegado)
#    Paso B: regex general
# ================================================

# ---- Paso A: Override para SUB-PLANEACION pegado o con separadores
# Captura número (1-4 dígitos) + etiqueta SUB[-_ ]?PLANEACION pegada o separada
pat_subplan = re.compile(
    r"^\s*(\d{1,4})\s*[-_]*\s*"
    r"(SUBG?(?:ERENCIA)?[-_ ]?PLANEAC(?:ION)?)\b",
    re.IGNORECASE)

extra_sub = desc.str.extract(pat_subplan, expand=True)
num_sub = extra_sub[0].astype("string")
tag_sub = extra_sub[1].astype("string")

# Inicializa columnas con NA
SECOP_ATENEA_TOTAL["Número_PAA_descripción"] = pd.Series(pd.NA, index=SECOP_ATENEA_TOTAL.index, dtype="string")
SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"] = pd.Series(pd.NA, index=SECOP_ATENEA_TOTAL.index, dtype="string")

# Donde matchea SUB-PLANEACION => forzar SPL
mask_sub = tag_sub.notna() & (tag_sub.str.strip() != "")
SECOP_ATENEA_TOTAL.loc[mask_sub, "Número_PAA_descripción"] = num_sub[mask_sub].str.strip()
SECOP_ATENEA_TOTAL.loc[mask_sub, "PAA::Código_DependenciaSolicitante"] = "SPL"

# ---- Paso B: regex general para el resto (incluye pegados tipo 014DESPRESTAR, 022 GEPRESTAR)
GATILLOS = r"PRESTAR|APOYAR"   # agrega más si quieres
pat_general = re.compile(
    rf"^\s*(\d{{1,4}})\s*[-_]*\s*"
    rf"([A-ZÁÉÍÓÚÑ]{{2,5}}?)(?=(?:\s|[-_]|$|{GATILLOS}))",
    re.IGNORECASE)

extra_gen = desc.str.extract(pat_general, expand=True)

# Solo llenar donde NO se llenó en el override
mask_empty = SECOP_ATENEA_TOTAL["Número_PAA_descripción"].isna() & extra_gen[0].notna()
SECOP_ATENEA_TOTAL.loc[mask_empty, "Número_PAA_descripción"] = extra_gen.loc[mask_empty, 0].astype("string").str.strip()

mask_empty_cod = SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"].isna() & extra_gen[1].notna()
SECOP_ATENEA_TOTAL.loc[mask_empty_cod, "PAA::Código_DependenciaSolicitante"] = extra_gen.loc[mask_empty_cod, 1].astype("string")

# ================================================
# 6) NORMALIZAR + APLICAR ALIAS AL CÓDIGO EXTRAÍDO
# ================================================
cod_raw = (
    SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"]
    .astype("string")
    .str.upper()
    .str.strip())

cod_norm = (
    cod_raw
    .str.replace("\u00A0", " ", regex=False)
    .str.replace(r"\s+", "", regex=True)
    .str.replace("_", "-", regex=False))

# base: lo que capture el regex
SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"] = cod_raw

# aplicar alias en crudo y también sobre normalizado
SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"] = SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"].replace(alias_codigos)
SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"] = (
    SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"]
    .fillna(cod_norm)  # si quedó NA, intenta con cod_norm
    .replace(alias_codigos)
    .astype("string"))

# asegurar formato final
SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"] = (
    SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"]
    .astype("string")
    .str.strip()
    .str.upper())

# ================================================
# 6B) FALLBACK: EXTRAER CÓDIGO DESDE PAA::ID_PAA_SECOP
#     cuando no se detectó en PAA::descripcion
# ================================================

codigos = sorted(set(mapa_dep) | set(alias_codigos), key=len, reverse=True)

id_norm = (
    "_" +
    SECOP_ATENEA_TOTAL["PAA::ID_PAA_SECOP"]
        .astype(str)
        .str.upper()
        .str.replace(r"[^A-Z0-9]+", "_", regex=True)
        .str.strip("_") + "_")

pat = rf"_({'|'.join(codigos)})_"

extra = id_norm.str.extract(pat, expand=False)

mask = (
    SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"].isna()
    | (SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"].astype(str).str.strip() == ""))

SECOP_ATENEA_TOTAL.loc[mask, "PAA::Código_DependenciaSolicitante"] = (
    extra[mask].replace(alias_codigos))

# ================================================
# 7) CREAR ID_PAA_GA
# ================================================
SECOP_ATENEA_TOTAL["Número_PAA_descripción"] = (
    SECOP_ATENEA_TOTAL["Número_PAA_descripción"]
    .astype("string").fillna("").str.strip())

SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"] = (
    SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"]
    .astype("string").fillna("").replace("<NA>", "")
    .str.strip().str.upper())

num = SECOP_ATENEA_TOTAL["Número_PAA_descripción"]
cod = SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"]

SECOP_ATENEA_TOTAL["ID_PAA_GA"] = np.where(
    (num != "") & (cod != ""),
    num + " " + cod, "",)

# ================================================
# 8) MAPEAR A DEPENDENCIA SOLICITANTE
# ================================================
SECOP_ATENEA_TOTAL["PAA::DependenciaSolicitante"] = SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"].map(mapa_dep)

# ================================================
# 9) LISTAR CÓDIGOS SIN MAPEAR (ya después de alias)
# ================================================
faltantes = sorted(
    set(SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"].dropna()) - set(mapa_dep.keys()))
print("\nCódigos de dependencia sin mapear (para revisar):", faltantes)


Códigos de dependencia sin mapear (para revisar): []


In [ ]:
################################################################
###========= PAA::ID_PAA_SECOP NUEVO FORMATO 2026 ===========###
################################################################

# ------------------------------------------------------------
# 1) ASEGURAR COLUMNAS DE SALIDA (viejas + nuevas)
#    (Para años != 2026 quedarán vacías en las nuevas)
# ------------------------------------------------------------
cols_out = [
    "Número_PAA_descripción",
    "PAA::Código_DependenciaSolicitante",
    "ID_PAA_GA",
    "PAA::DependenciaSolicitante",
    "PAA::Número_Proyecto",
    "PAA::Nombre_Proyecto",
    "PAA::Código_Meta",
    "PAA::Descripción_Meta",
    "PAA::Código_PosiciónPresupuestal",
    "PAA::Descripción_PosiciónPresupuestal",
    "PAA::Código_TipoContrato",
    "PAA::TipoContrato",
]
for c in cols_out:
    if c not in SECOP_ATENEA_TOTAL.columns:
        SECOP_ATENEA_TOTAL[c] = ""

# ------------------------------------------------------------
# 2) PARSEO 2026 (nuevo formato)
#    Ej: "8029_5_85940 _SGF_PSP_002 PRESTAR LOS..."
#
# Capturas:
#  - Proyecto: 8029
#  - Meta (número): 5  => Código_Meta: 8029_5
#  - Pospre: 85940
#  - Dependencia: SGF
#  - TipoContrato: PSP
#  - Número_PAA_descripción: 002
# ------------------------------------------------------------
pat_2026 = re.compile(
    r"^\s*(\d+)\s*_\s*(\d+)\s*_\s*(\d+)\s*[_\s]*([A-ZÁÉÍÓÚÑ]{2,6})\s*_\s*([A-Z]{2,6})\s*_\s*(\d+)(?=\D|$)",
    re.IGNORECASE)

mask_2026 = SECOP_ATENEA_TOTAL["AÑO_INFERIDO"].astype(str).str.strip().eq("2026")

extra_2026 = desc.where(mask_2026, "").str.extract(pat_2026, expand=True)
# 0: proyecto, 1: meta_num, 2: pospre, 3: dep, 4: tc, 5: paa_num

proyecto = extra_2026[0].astype("string")
meta_num = extra_2026[1].astype("string")
pospre_raw = extra_2026[2].astype("string")
dep_raw = extra_2026[3].astype("string").str.upper().str.strip()
tc_raw = extra_2026[4].astype("string").str.upper().str.strip()
paa_num = extra_2026[5].astype("string")

# Normalizar dependencia con alias
dep_norm = (
    dep_raw
    .str.replace(r"\s+", "", regex=True)
    .str.replace("_", "-", regex=False)
    .replace(alias_codigos)
)

# Número_PAA_descripción en 2026 (ej 002)
SECOP_ATENEA_TOTAL.loc[mask_2026, "Número_PAA_descripción"] = paa_num.fillna("").str.zfill(3)

# Código dependencia solicitante en 2026
SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Código_DependenciaSolicitante"] = dep_norm.fillna("")

# ID_PAA_GA en 2026: "002 SGF"
num_2026 = SECOP_ATENEA_TOTAL.loc[mask_2026, "Número_PAA_descripción"].fillna("").astype("string").str.strip()
dep_2026 = SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Código_DependenciaSolicitante"].fillna("").astype("string").str.strip()

SECOP_ATENEA_TOTAL.loc[mask_2026, "ID_PAA_GA"] = np.where(
    (num_2026 != "") & (dep_2026 != ""),
    num_2026 + " " + dep_2026,
    "",)

# Dependencia solicitante (nombre largo)
SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::DependenciaSolicitante"] = (
    SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Código_DependenciaSolicitante"].map(mapa_dep))

# ==========================================================
# FALLBACK DEPENDENCIA DESDE PAA::ID_PAA_SECOP
# (solo donde quedó vacía)
# ==========================================================

codigos = sorted(set(mapa_dep) | set(alias_codigos), key=len, reverse=True)

id_norm = ("_" +
    SECOP_ATENEA_TOTAL["PAA::ID_PAA_SECOP"]
        .astype(str)
        .str.upper()
        .str.replace(r"[^A-Z0-9]+", "_", regex=True)
        .str.strip("_") + "_")

extra = id_norm.str.extract(rf"_({'|'.join(codigos)})_", expand=False).replace(alias_codigos)

mask_fallback = (
    mask_2026 &
    (SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"].isna() |
        (SECOP_ATENEA_TOTAL["PAA::Código_DependenciaSolicitante"].astype(str).str.strip() == "")))

SECOP_ATENEA_TOTAL.loc[mask_fallback, "PAA::Código_DependenciaSolicitante"] = extra[mask_fallback]

SECOP_ATENEA_TOTAL.loc[mask_fallback, "PAA::DependenciaSolicitante"] = (
    SECOP_ATENEA_TOTAL.loc[mask_fallback, "PAA::Código_DependenciaSolicitante"].map(mapa_dep))

# NUEVAS columnas 2026:
SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Número_Proyecto"] = proyecto.fillna("").astype("string")

codigo_meta = np.where(
    (proyecto.fillna("") != "") & (meta_num.fillna("") != ""),
    (proyecto.fillna("").astype("string") + "_" + meta_num.fillna("").astype("string")),
    ""
)
SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Código_Meta"] = pd.Series(codigo_meta, index=SECOP_ATENEA_TOTAL.index).where(mask_2026, "")

# Pospre: guardar código como string; y para cruce usar últimos 6 dígitos (si tiene 5, se zfill a 6)
pospre_last6 = pospre_raw.fillna("").astype("string").str.replace(r"\D+", "", regex=True).str.zfill(6).str[-6:]
SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Código_PosiciónPresupuestal"] = pospre_last6

# Código tipo contrato
SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Código_TipoContrato"] = tc_raw.fillna("").astype("string")

# ------------------------------------------------------------
# 3) ENRIQUECER DESCRIPCIONES DESDE TABLAS AUXILIARES (solo 2026)
# ------------------------------------------------------------

# 3.1 Nombre y descripción de meta desde tab_proyectos
tabp = tab_proyectos.copy()

# Normalizar llaves
tabp["PROYECTO"] = tabp["PROYECTO"].astype(str).str.strip()
tabp["CODIGO_META"] = tabp["CODIGO_META"].astype(str).str.strip()

# Mapas
map_nombre_proy = dict(zip(tabp["PROYECTO"], tabp["NOMBRE_PROYECTO"]))
map_meta_ebi = dict(zip(tabp["CODIGO_META"], tabp["Meta_EBI"]))

SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Nombre_Proyecto"] = (
    SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Número_Proyecto"].astype(str).str.strip().map(map_nombre_proy)
)

SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Descripción_Meta"] = (
    SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Código_Meta"].astype(str).str.strip().map(map_meta_ebi)
)

# 3.2 Descripción pospre desde tab_Pospre (tomando últimos 6 números del código)
# --- construir mapa: last6 (zfill) -> descripción ---
keys_pospre = (tab_pospre["Pospre_2026"].astype(str)
               .str.replace("\u00A0","", regex=False)
               .str.replace(r"\D+","", regex=True)
               .str.zfill(6).str[-6:])

map_pospre = dict(zip(keys_pospre, tab_pospre["descripción_Pospre"]))

# --- llaves desde SECOP (k1 y alternativa k2) ---
k1 = (SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Código_PosiciónPresupuestal"].astype(str)
      .str.replace("\u00A0","", regex=False)
      .str.replace(r"\D+","", regex=True)
      .str.zfill(6).str[-6:])

k2 = "8" + k1.str[-5:]

SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Descripción_PosiciónPresupuestal"] = (
    k1.map(map_pospre).fillna(k2.map(map_pospre)))

# 3.3 Tipo contrato desde tab_tc
tct = tab_tc.copy()
tct["Codigo_TC"] = tct["Codigo_TC"].astype(str).str.strip().str.upper()
map_tc = dict(zip(tct["Codigo_TC"], tct["Tipo_Contrato"]))

SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::TipoContrato"] = (
    SECOP_ATENEA_TOTAL.loc[mask_2026, "PAA::Código_TipoContrato"].astype(str).str.strip().str.upper().map(map_tc))

# SECOP_ATENEA_TOTAL.drop(columns=["Número_PAA_descripción"], inplace=True)

## Reemplazar vacíos "", " " por NA:
SECOP_ATENEA_TOTAL = SECOP_ATENEA_TOTAL.replace(r"^\s*$", pd.NA, regex=True)

In [ ]:
# SECOP_ATENEA_TOTAL[SECOP_ATENEA_TOTAL["AÑO_INFERIDO"]==2025][['Número_PAA_descripción','PAA::ID_PAA_SECOP','PAA::Código_DependenciaSolicitante',
#        'ID_PAA_GA', 'PAA::DependenciaSolicitante', 'Número_PAA_descripción',
#        'PAA::Número_Proyecto', 'PAA::Nombre_Proyecto', 'PAA::Código_Meta',
#        'PAA::Descripción_Meta', 'PAA::Código_PosiciónPresupuestal',
#        'PAA::Descripción_PosiciónPresupuestal', 'PAA::Código_TipoContrato',
#        'PAA::TipoContrato']].isnull().sum()  ####--- 29 faltantes por estructura extraña en PAA::ID_PAA_SECOP

## **Tabla Maestra (estado actual)**

Corresponde a una tabla consolidada que integra exclusivamente las variables seleccionadas con base en los siguientes criterios:

+ Variables requeridas para reportes oficiales.
+ Requerimientos definidos por el equipo de Gestión Corporativa.
+ Pertinencia analítica y viabilidad técnica.
+ Relevancia funcional y estratégica para la toma de decisiones.

La estructura contempla una única fila por contrato, incorporando únicamente la versión más reciente y vigente de cada registro, garantizando consistencia, trazabilidad y actualización de la información.

Nombre de la hoja: *ATENEA_SEL*

In [ ]:
SECOP_vars = [
    # 1️. Identificación del contrato
    'id_contrato',
    'referencia_del_contrato (Contratos_electronicos)',
    'descripcion_del_proceso (Contratos_electronicos)',
    'AÑO_INFERIDO',
    'urlproceso (Contratos_electronicos)',

    # 2️. Estado del contrato
    'estado_contrato (Contratos_electronicos)',
    'ESTADO VIGENTE_inferido',
    'liquidaci_n (Contratos_electronicos)',

    # 3️. Fechas clave
    'fecha_de_firma (Contratos_electronicos)',
    'fecha_de_inicio_del_contrato (Contratos_electronicos)',
    'fecha_de_fin_del_contrato (Contratos_electronicos)',

    # 4️. Plazos
    'duraci_n_del_contrato (Contratos_electronicos)',
    'PLAZO DE EJECUCIÓN EN MESES (INICIAL)_inferido',
    'PLAZO DE EJECUCIÓN FINAL DEL CONTRATO (DÍAS)_inferido',
    'dias_adicionados (Contratos_electronicos)',
    'porc_avanceplazo_inferido',

    # 5️. Valores y recursos
    'valor_del_contrato (Contratos_electronicos)',
    'valor_pago_mensual_inferido',
    'valor_pagado (Contratos_electronicos)',
    'origen_de_los_recursos (Contratos_electronicos)',
    'porc_ejecucion_financiera',
    'recursos_propios (Contratos_electronicos)',
    'destino_gasto (Contratos_electronicos)',

    # 6️. Tipo y modalidad contractual
    'tipo_de_contrato (Contratos_electronicos)',
    'modalidad_de_contratacion (Contratos_electronicos)',
    'justificacion_modalidad_de (Contratos_electronicos)',

    # 7️. Información del proveedor
    'tipodocproveedor (Contratos_electronicos)',
    'documento_proveedor (Contratos_electronicos)',
    'proveedor_adjudicado (Contratos_electronicos)',
    'DígitoVerificaciónDIAN_DV',
    'nacionalidad_representante_legal (Contratos_electronicos)',
    'g_nero_representante_legal (Contratos_electronicos)',
    'TIPO_PERSONA_INFERIDO',
    'NATURALEZA_INFERIDO',
    'CLASIFICACIÓN_INFERIDO',

    # 8️. Información del Supervisor
    'nombre_supervisor (Contratos_electronicos)',
    'tipo_de_documento_supervisor (Contratos_electronicos)',
    'n_mero_de_documento_supervisor (Contratos_electronicos)',
    'DígitoVerificaciónDIAN_DV_supervisor',

    # 9. Trazabilidad del contrato
    'TERMINACIÓN ANTICIPADA_inferido',
    'SUSPENSIÓN_INFERIDO',
    'CESIÓN_inferido',
    'identificador_modificacion (modificaciones)',
    'estado_modificacion (modificaciones)',
    'tipo_modificacion',
    'fecha_de_aprobacion (modificaciones)',
    'fecha_fin_liquidacion (modificaciones)',
    'ultima_actualizacion (contratos_electronicos)',
    'OBSERVACIONES_inferido',

    # 10. Vinculación PAA (planeación)
    'ID_PAA_GA',
    'Número_PAA_descripción',
    'PAA::ID_PAA_SECOP',
    'PAA::procesos_relacionados',
    'PAA::Código_DependenciaSolicitante',
    'PAA::DependenciaSolicitante',
    'PAA::Número_Proyecto',
    'PAA::Nombre_Proyecto',
    'PAA::Código_Meta',
    'PAA::Descripción_Meta',
    'PAA::Código_PosiciónPresupuestal',
    'PAA::Descripción_PosiciónPresupuestal',
    'PAA::Código_TipoContrato',
    'PAA::TipoContrato',
    'PAA::requiere_vigencias_futuras'
]

# Crear copia de la base de datos SECOP_ATENEA_TOTAL
SECOP_ATENEA_SEL = SECOP_ATENEA_TOTAL.reindex(columns=SECOP_vars).copy()

In [ ]:
# ============================================================
# 3.1) Homologaciones y Limpieza Variables a comparar. Pt.1
# ============================================================

#------ Número de Proceso PAA
s2 = SECOP_ATENEA_SEL["PAA::procesos_relacionados"].astype("string")
s2 = (s2.str.upper().str.replace(r"\bATENA\b", "ATENEA", regex=True)
     .str.replace(".", "", regex=False)
     .str.replace(",", "", regex=False)
     .str.replace(r"\s+", " ", regex=True).str.strip())
# 1) Si viene por ejemplo "ATENEA 243-2025" o "ATENEA 067-2024" => "ATENEA-243-2025"
s2 = s2.str.replace(r"^ATENEA\s+(?=\d)", "ATENEA-", regex=True)
# 2) Normaliza guiones (espacios alrededor) => "243 - 2025" -> "243-2025"
s2 = s2.str.replace(r"\s*-\s*", "-", regex=True)
# 3) Si viene por ejemplo "349-2026" sin prefijo => "ATENEA-349-2026"
s2 = s2.str.replace(r"^(?=\d{1,4}-\d{4}$)", "ATENEA-", regex=True)
# Asignación
SECOP_ATENEA_SEL["PAA::procesos_relacionados"] = s2

#------ Variables Tipo fecha

cols_fechas = [
    "fecha_de_firma (Contratos_electronicos)",
    "fecha_de_inicio_del_contrato (Contratos_electronicos)",
    "fecha_de_fin_del_contrato (Contratos_electronicos)",
    "fecha_de_aprobacion (modificaciones)",
    "fecha_fin_liquidacion (modificaciones)",
    "ultima_actualizacion (contratos_electronicos)"
]

SECOP_ATENEA_SEL[cols_fechas] = SECOP_ATENEA_SEL[cols_fechas].apply(
    lambda x: pd.to_datetime(x, errors="coerce"))


#------ Duración del contrato, Propuesta 1, formulado como la diferencia entre fecha_fin y fecha_inicio:
SECOP_ATENEA_SEL["duraci_n_del_contrato (Contratos_electronicos)_formulado"] = SECOP_ATENEA_SEL["fecha_de_fin_del_contrato (Contratos_electronicos)"] - SECOP_ATENEA_SEL["fecha_de_inicio_del_contrato (Contratos_electronicos)"]
SECOP_ATENEA_SEL["duraci_n_del_contrato (Contratos_electronicos)_formulado"] = pd.to_timedelta(SECOP_ATENEA_SEL["duraci_n_del_contrato (Contratos_electronicos)_formulado"], errors="coerce")
SECOP_ATENEA_SEL["duraci_n_del_contrato (Contratos_electronicos)_formulado"] = (SECOP_ATENEA_SEL["duraci_n_del_contrato (Contratos_electronicos)_formulado"].dt.days).astype("Int64")

#------ Duración del contrato, Propuesta 2, estandarización de registros originales:
def convertir_a_dias(valor):
    if pd.isna(valor):
        return np.nan
    s = str(valor).strip()
    m = re.search(r"(\d+)", s)
    if not m:
        return np.nan
    n = int(m.group(1))
    s_low = s.lower()
    if "dia" in s_low or "d\u00eda" in s_low:
        return n
    if "mes" in s_low:
        return int(round(n * 365/12))  # 30.4167 días/mes aprox.
    if "año" in s_low or "ano" in s_low:
        return n * 365
    return np.nan

SECOP_ATENEA_SEL["duraci_n_del_contrato (Contratos_electronicos)"] = SECOP_ATENEA_SEL["duraci_n_del_contrato (Contratos_electronicos)"].apply(convertir_a_dias)

In [ ]:
# ============================================================
# 3.2) Homologaciones y Limpieza Variables a comparar. Pt.2
# ============================================================

#------ Tipos de documento
cols_tipo_doc = [
    "tipodocproveedor (Contratos_electronicos)",
    "tipo_de_documento_supervisor (Contratos_electronicos)"]

reemplazos = {
    "CEDULA DE CIUDADANIA": "CC",
    "PERMISO POR PROTECCION TEMPORAL": "CE"}

for col in cols_tipo_doc:
    if col in SECOP_ATENEA_SEL.columns:
        SECOP_ATENEA_SEL[col] = (
            SECOP_ATENEA_SEL[col]
            .astype("string")
            .fillna("")
            .str.upper()
            .str.strip()
            .apply(quitar_tildes)
            .replace(reemplazos))

#------ Género del contratista

# Columnas
col_nom = "proveedor_adjudicado (Contratos_electronicos)"
col_gen = "g_nero_representante_legal (Contratos_electronicos)"

# - 1) Genero representante legal: HOMBRE/MUJER -> M/F
if col_gen in SECOP_ATENEA_SEL.columns:
    SECOP_ATENEA_SEL[col_gen] = (
        SECOP_ATENEA_SEL[col_gen].astype("string").fillna("").str.upper().str.strip().apply(quitar_tildes)
        .replace({"HOMBRE": "M", "MUJER": "F"}))
else:
    SECOP_ATENEA_SEL[col_gen] = pd.Series("", index=SECOP_ATENEA_SEL.index, dtype="string")

# - 2) Nombres en mayúsculas y sin tildes para mayor merge con la lista
SECOP_ATENEA_SEL[col_nom] = (SECOP_ATENEA_SEL[col_nom].astype("string").fillna("").str.upper().apply(quitar_tildes))

# - 3) Solo inferir donde está vacío / no definido / otro
mask = SECOP_ATENEA_SEL[col_gen].isin(["", "NO DEFINIDO", "OTRO"])

# Si no hay nada que inferir, salir rápido
if mask.any():
    # - 4) Diccionario NOMBRE -> GENERO
    lista = Lista_Nombres_Genero.copy()
    lista["NOMBRE"] = lista["NOMBRE"].astype(str).str.strip().str.upper()
    lista["GENERO"] = lista["GENERO"].astype(str).str.strip().str.upper()

    mapa_genero = dict(zip(lista["NOMBRE"], lista["GENERO"]))

    # - 5) Regex único con todos los nombres
    nombres = lista["NOMBRE"].dropna().unique().tolist()
    nombres = sorted(nombres, key=len, reverse=True)  # largo->corto

    pat = re.compile(r"\b(" + "|".join(map(re.escape, nombres)) + r")\b")

    # - 6) Encontrar todos los nombres por fila (solo en mask)
    matches = SECOP_ATENEA_SEL.loc[mask, col_nom].str.findall(pat)

    # - 7) Decidir mayoría
    def decide_genero(lista_nombres):
        if not lista_nombres:
            return ""
        gens = [mapa_genero.get(n, "") for n in lista_nombres]
        m = sum(g == "M" for g in gens)
        f = sum(g == "F" for g in gens)
        if f > m:
            return "F"
        if m > f:
            return "M"
        return ""

    inferido = matches.apply(decide_genero)

    # - 8) Rellenar solo donde aplica
    SECOP_ATENEA_SEL.loc[mask, col_gen] = inferido.values

# - 9) Vaciar género cuando el proveedor es persona jurídica (NIT)
SECOP_ATENEA_SEL.loc[
    SECOP_ATENEA_SEL["tipodocproveedor (Contratos_electronicos)"].astype(str).str.upper() == "NIT",
    "g_nero_representante_legal (Contratos_electronicos)"] = np.nan

In [ ]:
# ============================================================
# 3.3) Homologaciones y Limpieza Variables a comparar. Pt.3
# ============================================================

#------ Modalidad de contratación
def limpiar_texto(s): return (s.astype(str).str.strip().str.lower().str.replace(r"\s+", " ", regex=True))

SECOP_ATENEA_SEL["modalidad_de_contratacion (Contratos_electronicos)"] = limpiar_texto(
    SECOP_ATENEA_SEL["modalidad_de_contratacion (Contratos_electronicos)"])

SECOP_ATENEA_SEL["modalidad_de_contratacion (Contratos_electronicos)"] = (
    SECOP_ATENEA_SEL["modalidad_de_contratacion (Contratos_electronicos)"]
    .replace({
    "contratación directa": "5 Contratación directa",
    "contratación directa (con ofertas)": "5 Contratación directa",
    "contratación régimen especial": "8 Otra Regimen Especial",
    "contratación régimen especial (con ofertas)": "8 Otra Regimen Especial",
    "selección abreviada de menor cuantía": "2 Selección abreviada",
    "selección abreviada subasta inversa": "2 Selección abreviada",
    "mínima cuantía": "4 Mínima cuantía",
    "concurso de méritos abierto": "3 Concurso de méritos"
}))

#------ Tipo de contrato
SECOP_ATENEA_SEL["tipo_de_contrato (Contratos_electronicos)"] = limpiar_texto(
    SECOP_ATENEA_SEL["tipo_de_contrato (Contratos_electronicos)"])

SECOP_ATENEA_SEL["tipo_de_contrato (Contratos_electronicos)"] = (
    SECOP_ATENEA_SEL["tipo_de_contrato (Contratos_electronicos)"]
    .replace({
    "prestación de servicios": "31 31-Servicios Profesionales",
    "consultoría": "29 29-Consultoría (Otros)",
    "suministros": "42 42-Suministro de Bienes en general",
    "compraventa": "121 121-Compraventa (Bienes Muebles)",
    "arrendamiento de inmuebles": "132 132-Arrendamiento de bienes inmuebles",
    "arrendamiento de muebles": "131 131-Arrendamiento de bienes muebles",
    "seguros": "72 72-Contrato de Seguros",
    "comodato": "904 904-Comodato",
    "otro": "999 999-Otro tipo de naturaleza de contratos"
}))

#------ Justificación modalidad de contrato
SECOP_ATENEA_SEL["justificacion_modalidad_de (Contratos_electronicos)"] = limpiar_texto(
    SECOP_ATENEA_SEL["justificacion_modalidad_de (Contratos_electronicos)"])

SECOP_ATENEA_SEL["justificacion_modalidad_de (Contratos_electronicos)"] = (
    SECOP_ATENEA_SEL["justificacion_modalidad_de (Contratos_electronicos)"]
    .replace({
    "servicios profesionales y apoyo a la gestión": "33 Prestación de Servicios Profesionales y Apoyo (5-8)",
    "contratos o convenios interadministrativos (con valor)": "13 Contratos Interadministrativos (5-8)",
    "contratos o convenios interadministrativos (valor cero)": "13 Contratos Interadministrativos (5-8)",
    "presupuesto menor al 10% de la menor cuantía": "30 Porcentaje Mínima Cuantía (4)",
    "presupuesto inferior al 10% de la menor cuantía": "30 Porcentaje Mínima Cuantía (4)",
    "contratos para el desarrollo de actividades científicas y tecnológicas": "17 Desarrollo Actividad Científica y Tecnológica (5-8)",
    "arrendamiento de inmuebles": "6 Arrendamientos y Adquisición de Inmuebles (5-8)",
    "suministro de bienes y servicios de características técnicas uniformes y común utilización":
        "4 Adquisión o Suministro de Bienes y Servicios de Carácterísticas Técnicas Uniformes y de Común Utilización (Procedimiento: Siubasta Inversa, Acuerdo Marco de Precios, Bolsa de Productos) (2)",
    "no existe pluralidad de oferentes en el mercado": "38 Sin Pluralidad de Oferentes (5-8)",
    "prestamo de uso": "8 Comodatos (5)"
}))


#------ Estado de modificación
SECOP_ATENEA_SEL["estado_modificacion (modificaciones)"] = (
    SECOP_ATENEA_TOTAL["estado_modificacion (modificaciones)"]
    .str.strip()
    .str.lower()
    .str.capitalize())

#------ Identificador de modificación
# Extraer el número final
SECOP_ATENEA_SEL["identificador_modificacion (modificaciones)"] = (
    SECOP_ATENEA_SEL["identificador_modificacion (modificaciones)"]
    .str.extract(r'CTRMOD\.(\d+)', expand=False)
    .astype("string"))

In [ ]:
# ============================================================
# 3.4) Homologaciones y Limpieza Variables a comparar. Pt.4
# ============================================================

#------ Estado del contrato y de modificación
cols_estado = [
    "estado_contrato (contratos_electronicos)",
    "estado_modificacion (modificaciones)"]

SECOP_ATENEA_SEL[cols_estado] = (
    SECOP_ATENEA_TOTAL[cols_estado]
        .apply(lambda x: x.str.strip().str.lower().str.capitalize()))

#------ Identificador de modificación
# Extraer el número final
SECOP_ATENEA_SEL["identificador_modificacion (modificaciones)"] = (
    SECOP_ATENEA_SEL["identificador_modificacion (modificaciones)"]
    .str.extract(r'CTRMOD\.(\d+)', expand=False)
    .astype("string"))

#------ Tipo de gasto
SECOP_ATENEA_SEL["destino_gasto (Contratos_electronicos)"] = (SECOP_ATENEA_SEL["destino_gasto (Contratos_electronicos)"].replace({
    "Inversión": "1 1. Inversión", "Funcionamiento": "2 2. Funcionamiento",
    "Servicio de la deuda": "3 3. Servicio de la deuda", "Otro": "4 4. Otro"}))

#------ Tipo de persona
SECOP_ATENEA_SEL["TIPO_PERSONA_INFERIDO"] = (SECOP_ATENEA_SEL["TIPO_PERSONA_INFERIDO"].replace({
    "Jurídica": "2 Jurídica", "Natural": "1 Natural"}))

#------ Naturaleza
SECOP_ATENEA_SEL["NATURALEZA_INFERIDO"] = (SECOP_ATENEA_SEL["NATURALEZA_INFERIDO"].replace({
    "Privada": "2 Privada (1)",
    "Pública": "3 Pública (2-3)"}))


#------ Clasificación
SECOP_ATENEA_SEL["CLASIFICACIÓN_INFERIDO"] = (SECOP_ATENEA_SEL["CLASIFICACIÓN_INFERIDO"].replace({
    "Persona Natural": "4 Persona Natural (2)",
    "Privadas": "3 Privadas (2)",
    "Universidades": "14 Universidades (4)",
    "Públicos": "9 Públicos (3)"
}))

#------ Variables de SI/NO
cols_si_no = [
    "SUSPENSIÓN_INFERIDO",
    "CESIÓN_inferido",
    "ESTADO VIGENTE_inferido",
    "TERMINACIÓN ANTICIPADA_inferido",
    "PAA::requiere_vigencias_futuras",
    "liquidaci_n (contratos_electronicos)",
    "prorroga_inferido"]

SECOP_ATENEA_SEL[cols_si_no] = (
    SECOP_ATENEA_SEL[cols_si_no]
        .apply(lambda x: x.str.strip().str.upper())
        .replace({"SI": "1 SI", "NO": "2 NO"}))

#------ URL proceso
SECOP_ATENEA_SEL["urlproceso (Contratos_electronicos)"] = (
    SECOP_ATENEA_SEL["urlproceso (Contratos_electronicos)"]
    .apply(lambda x: x.get("url") if isinstance(x, dict)
           else ast.literal_eval(x).get("url") if isinstance(x, str)
           else x))

#------ Nacionalidad Representante legal
SECOP_ATENEA_SEL["nacionalidad_representante_legal (Contratos_electronicos)"] = SECOP_ATENEA_SEL["nacionalidad_representante_legal (Contratos_electronicos)"].replace({'CO':'1 NACIONAL', 'ES':'2 EXTRANJERO'})

#------ Número de documento del supervisor
SECOP_ATENEA_SEL["n_mero_de_documento_supervisor (Contratos_electronicos)"] = SECOP_ATENEA_SEL["n_mero_de_documento_supervisor (Contratos_electronicos)"].astype(str).str.replace(".", "", regex=False)

#------ DEPENDENCIA DE SOLICITANTE
SECOP_ATENEA_SEL["PAA::DependenciaSolicitante"] = SECOP_ATENEA_SEL["PAA::DependenciaSolicitante"].replace({
    "SUB. GESTIÓN_ADMINISTRATIVA": "SUBGERENCIA DE GESTIÓN ADMINISTRATIVA",
    "OFICINA JURÍDICA": "OFICINA ASESORA JURÍDICA",
    "SUBGERENCIA ADMINISTRATIVA": "SUBGERENCIA DE GESTIÓN ADMINISTRATIVA"})

## **Descargar el Excel a tu PC**

In [ ]:
# Cambiar orden de variables de duración
col_target = 'duraci_n_del_contrato (Contratos_electronicos)'
col_move = 'duraci_n_del_contrato (Contratos_electronicos)_formulado'

SECOP_ATENEA_SEL.insert(SECOP_ATENEA_SEL.columns.get_loc(col_target) + 1, col_move, SECOP_ATENEA_SEL.pop(col_move))

# Pasar a minúscula todos los nombres de las variables
SECOP_ATENEA_TOTAL.columns = SECOP_ATENEA_TOTAL.columns.str.lower()
SECOP_ATENEA_SEL.columns = SECOP_ATENEA_SEL.columns.str.lower()

In [ ]:
# Eliminar Duplicados
SECOP_ATENEA_SEL = (SECOP_ATENEA_SEL
    .sort_values("referencia_del_contrato (contratos_electronicos)")
    .drop_duplicates(subset=["referencia_del_contrato (contratos_electronicos)"], keep="first")
    .reset_index(drop=True))

# Guardar ambos DataFrames en un mismo libro Excel
with pd.ExcelWriter(EXCEL_FILE) as writer:
    SECOP_ATENEA_TOTAL.to_excel( writer,  sheet_name="SECOP_ATENEA_TOTAL",index=False)
    SECOP_ATENEA_SEL.to_excel(writer,sheet_name="SECOP_ATENEA_SEL",index=False)

# Descarga el archivo final SECOP_ATENEA_TOTAL.xlsx
files.download(EXCEL_FILE)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>